In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('satellite\data\merged_kayrros_mars_carbonmapper_11032025_coal.csv')
df = df.sort_values(by=["country","date"])

df

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\rebek\AppData\Local\Temp\ipykernel_47688\1067397650.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv('satellite\data\merged_kayrros_mars_carbonmapper_11032025_coal.csv')


,sensor,country,sector,date,emission rate (kg/h),uncertainty (kg/h),latitude,longitude,comments,platform
0,Sentinel-5P/TROPOMI - ESA,Australia,Coal,01/05/2020,31330.000000,14100.000000,-22.841200,148.657400,Downloaded from https://methanewatch.kayrros.c...,Kayrros Methane Watch
1,Sentinel-5P/TROPOMI - ESA,Australia,Coal,01/09/2024,14600.000000,6570.000000,-21.940500,148.031200,Downloaded from https://methanewatch.kayrros.c...,Kayrros Methane Watch
2,Sentinel-5P/TROPOMI - ESA,Australia,Coal,01/11/2023,5090.000000,2290.000000,-21.861500,148.320900,Downloaded from https://methanewatch.kayrros.c...,Kayrros Methane Watch
3,Sentinel-5P/TROPOMI - ESA,Australia,Coal,02/09/2024,16370.000000,7370.000000,-22.969800,148.862000,Downloaded from https://methanewatch.kayrros.c...,Kayrros Methane Watch
4,Sentinel-5P/TROPOMI - ESA,Australia,Coal,03/06/2022,12380.000000,5570.000000,-22.076700,147.929300,Downloaded from https://methanewatch.kayrros.c...,Kayrros Methane Watch
...,...,...,...,...,...,...,...,...,...,...
1720,EMIT - NASA,United States of America,Coal,30/05/2023,4780.000000,2380.000000,39.008200,-78.342000,Downloaded from https://methanewatch.kayrros.c...,Kayrros Methane Watch
1721,Global Airborne Observatory,United States of America,Coal,30/07/2020,469.105365,194.767074,40.570045,-112.079413,Downloaded from https://data.carbonmapper.org/...,Carbon Mapper
1722,Global Airborne Observatory,United States of America,Coal,31/07/2020,421.154408,173.447031,39.684543,-111.157220,Downloaded from https://data.carbonmapper.org/...,Carbon Mapper
1723,Global Airborne Observatory,United States of America,Coal,31/07/2020,153.349390,49.317373,39.682895,-110.545248,Downloaded from https://data.carbonmapper.org/...,Carbon Mapper


In [3]:
# Step 1: Identify duplicates based on sensor, date, lat, lon
dup_keys = ["sensor", "date", "latitude", "longitude"]

# Step 2: Keep only those groups where platform differs
df['platform_count'] = df.groupby(dup_keys)['platform'].transform('nunique')

# Step 3: Filter only rows with >1 unique platform
df_dupes = df[df['platform_count'] > 1].copy()

# Step 4: Define a helper to pick the best record from each group
def pick_best(group):
    # Prefer rows with both emission rate and uncertainty not null
    has_both = group[
        group['emission rate (kg/h)'].notna() &
        group['uncertainty (kg/h)'].notna()
    ]
    if not has_both.empty:
        return has_both.iloc[0]  # keep the first of those
    else:
        return group.iloc[0]     # otherwise keep the first in the group

# Step 5: Apply per-group selection
result_dupes = df_dupes.groupby(dup_keys, group_keys=False).apply(pick_best)

# Step 6: Combine with the rest of the data (non-duplicates)
result = pd.concat([
    df[df['platform_count'] == 1],  # unique platform
    result_dupes
], ignore_index=True)

# Step 7: Drop the helper column
result = result.drop(columns='platform_count')

result = result.sort_values(by=["country","date"])

C:\Users\rebek\AppData\Local\Temp\ipykernel_47688\1187132962.py:23: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_dupes = df_dupes.groupby(dup_keys, group_keys=False).apply(pick_best)


In [4]:
result.to_csv('final_data.csv', index=False)